# Explore: Heart Rate During the Mock Audition

Use this notebook to explore the HR data interactively. Select sessions, toggle subjects, adjust smoothing, and zoom into specific phases.

**To get started:** Run all cells (Runtime → Run all), then use the controls below each figure.

In [ ]:
# Setup — run this cell first
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

try:
    import ipywidgets as widgets
except ImportError:
    !pip install ipywidgets -q
    import ipywidgets as widgets

# Load data — update these paths if running outside the archive
hr = pd.read_csv('data/aligned_hr.csv')
phase_sum = pd.read_csv('data/phase_summaries.csv')

# Metadata — anonymized
singer_info = {
    'S1': ('Singer 1', 'M'),
    'S2': ('Singer 2', 'F'),
    'S3': ('Singer 3', 'F'),
    'S4': ('Singer 4', 'F'),
    'S5': ('Singer 5', 'M'),
}

phase_labels = {
    'pre_singing': 'Pre-Singing',
    'song_1': 'Song 1 (Prepared)',
    'inter_song_gap': 'Inter-Song Gap',
    'song_2': 'Song 2 (Assigned)',
    'post_session': 'Post-Session'
}

phase_colors = {
    'pre_singing': 'rgba(255, 193, 7, 0.15)',
    'song_1': 'rgba(76, 175, 80, 0.15)',
    'inter_song_gap': 'rgba(158, 158, 158, 0.15)',
    'song_2': 'rgba(33, 150, 243, 0.15)',
    'post_session': 'rgba(244, 67, 54, 0.15)'
}

phase_order = ['pre_singing', 'song_1', 'inter_song_gap', 'song_2', 'post_session']

subject_colors = {
    'singer': '#e74c3c',
    'F1': '#8e44ad',
    'F3': '#2980b9'
}

print('Data loaded:', len(hr), 'HR samples across', hr['session'].nunique(), 'sessions')
print('Phase summaries:', len(phase_sum), 'rows')

---
## Session Explorer

Pick a session, choose which subjects to show, and adjust the smoothing window. The smoothing slider applies a rolling average — higher values reveal the overall trajectory, lower values show beat-to-beat variation.

In [ ]:
# Session Explorer with interactive controls

session_dropdown = widgets.Dropdown(
    options=[(f"{singer_info[s][0]} ({singer_info[s][1]})", s) for s in ['S1','S2','S3','S4','S5']],
    value='S1',
    description='Session:'
)

show_singer = widgets.Checkbox(value=True, description='Singer', indent=False,
                               style={'description_width': 'initial'})
show_f1 = widgets.Checkbox(value=True, description='Panelist 1', indent=False,
                           style={'description_width': 'initial'})
show_f3 = widgets.Checkbox(value=True, description='Panelist 3', indent=False,
                           style={'description_width': 'initial'})

smoothing_slider = widgets.IntSlider(
    value=1, min=1, max=30, step=1,
    description='Smoothing (s):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

show_phases = widgets.Checkbox(value=True, description='Show phase shading', indent=False,
                               style={'description_width': 'initial'})

activation_gap_mode = widgets.Checkbox(
    value=False, description='Activation gap mode (singer minus panelist mean)', indent=False,
    style={'description_width': 'initial'}
)

fig_widget = go.FigureWidget()
fig_widget.update_layout(
    template='plotly_white', height=450,
    xaxis_title='Elapsed Time (seconds)',
    yaxis_title='Heart Rate (bpm)',
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=60, b=60)
)

def update_session_plot(*args):
    s = session_dropdown.value
    sess = hr[hr['session'] == s]
    window = smoothing_slider.value
    gap_mode = activation_gap_mode.value
    name = f"{singer_info[s][0]} ({singer_info[s][1]})"
    
    with fig_widget.batch_update():
        fig_widget.data = []
        fig_widget.layout.shapes = []
        fig_widget.layout.annotations = []
        
        # Phase shading
        if show_phases.value and not gap_mode:
            shapes = []
            annotations = []
            for phase in phase_order:
                pdata = sess[sess['phase'] == phase]
                if len(pdata) == 0:
                    continue
                x0 = float(pdata['elapsed_s'].min())
                x1 = float(pdata['elapsed_s'].max())
                shapes.append(dict(
                    type='rect', x0=x0, x1=x1, y0=0, y1=1,
                    yref='paper', fillcolor=phase_colors[phase],
                    layer='below', line_width=0
                ))
                annotations.append(dict(
                    x=x0 + 2, y=1, yref='paper',
                    text=phase_labels[phase], showarrow=False,
                    font=dict(size=9, color='#666'),
                    xanchor='left', yanchor='top'
                ))
            fig_widget.layout.shapes = shapes
            fig_widget.layout.annotations = annotations
        
        if gap_mode:
            # Compute singer HR minus panelist mean
            singer_data = sess[sess['subject'] == s].copy().set_index('elapsed_s')
            f1_data = sess[sess['subject'] == 'F1'].copy().set_index('elapsed_s')
            f3_data = sess[sess['subject'] == 'F3'].copy().set_index('elapsed_s')
            
            common_idx = singer_data.index.intersection(f1_data.index).intersection(f3_data.index)
            if len(common_idx) > 0:
                panelist_mean = (f1_data.loc[common_idx, 'hr_bpm'] + f3_data.loc[common_idx, 'hr_bpm']) / 2
                gap = singer_data.loc[common_idx, 'hr_bpm'] - panelist_mean
                if window > 1:
                    gap = gap.rolling(window, center=True, min_periods=1).mean()
                fig_widget.add_trace(go.Scatter(
                    x=list(common_idx), y=list(gap),
                    mode='lines', name='Activation Gap',
                    line=dict(color='#e74c3c', width=2),
                    hovertemplate='%{y:+.1f} bpm<extra>Gap</extra>'
                ))
            fig_widget.add_hline(y=0, line_dash='dash', line_color='black', line_width=1)
            fig_widget.layout.yaxis.title = 'Activation Gap (bpm)'
            fig_widget.layout.title = f'{name} — Activation Gap'
        else:
            # Standard traces
            if show_singer.value:
                sdata = sess[sess['subject'] == s].copy()
                y = sdata['hr_bpm']
                if window > 1:
                    y = y.rolling(window, center=True, min_periods=1).mean()
                fig_widget.add_trace(go.Scatter(
                    x=list(sdata['elapsed_s']), y=list(y),
                    mode='lines', name=name,
                    line=dict(color=subject_colors['singer'], width=2),
                    hovertemplate='%{y:.0f} bpm<extra>' + name + '</extra>'
                ))
            
            if show_f1.value:
                f1 = sess[sess['subject'] == 'F1'].copy()
                y = f1['hr_bpm']
                if window > 1:
                    y = y.rolling(window, center=True, min_periods=1).mean()
                fig_widget.add_trace(go.Scatter(
                    x=list(f1['elapsed_s']), y=list(y),
                    mode='lines', name='Panelist 1',
                    line=dict(color=subject_colors['F1'], width=1.5),
                    hovertemplate='%{y:.0f} bpm<extra>Panelist 1</extra>'
                ))
            
            if show_f3.value:
                f3 = sess[sess['subject'] == 'F3'].copy()
                y = f3['hr_bpm']
                if window > 1:
                    y = y.rolling(window, center=True, min_periods=1).mean()
                fig_widget.add_trace(go.Scatter(
                    x=list(f3['elapsed_s']), y=list(y),
                    mode='lines', name='Panelist 3',
                    line=dict(color=subject_colors['F3'], width=1.5),
                    hovertemplate='%{y:.0f} bpm<extra>Panelist 3</extra>'
                ))
            
            fig_widget.layout.yaxis.title = 'Heart Rate (bpm)'
            smooth_label = f' ({window}s avg)' if window > 1 else ''
            fig_widget.layout.title = f'{name}{smooth_label}'

# Wire up all controls
for w in [session_dropdown, show_singer, show_f1, show_f3, smoothing_slider, show_phases, activation_gap_mode]:
    w.observe(update_session_plot, 'value')

controls = widgets.VBox([
    session_dropdown,
    widgets.HBox([show_singer, show_f1, show_f3, show_phases]),
    smoothing_slider,
    activation_gap_mode
])

display(controls)
display(fig_widget)
update_session_plot()  # initial draw

---
## Phase Comparison

Select which phases and singers to compare. This lets you isolate specific moments — e.g., just the inter-song gaps, or just the assigned songs — across singers.

In [ ]:
# Phase comparison — filter by singer and phase

singer_select = widgets.SelectMultiple(
    options=[(f"{singer_info[s][0]} ({singer_info[s][1]})", s) for s in ['S1','S2','S3','S4','S5']],
    value=['S1','S2','S3','S4','S5'],
    description='Singers:',
    rows=5,
    style={'description_width': 'initial'}
)

phase_select = widgets.SelectMultiple(
    options=[(phase_labels[p], p) for p in phase_order],
    value=['pre_singing', 'song_1', 'song_2'],
    description='Phases:',
    rows=5,
    style={'description_width': 'initial'}
)

metric_dropdown = widgets.Dropdown(
    options=[('Mean HR', 'mean_hr'), ('SD', 'sd_hr'), ('Min HR', 'min_hr'), ('Max HR', 'max_hr')],
    value='mean_hr',
    description='Metric:',
    style={'description_width': 'initial'}
)

singer_plot_colors = {
    'S1': '#e74c3c',
    'S2': '#e67e22',
    'S3': '#27ae60',
    'S4': '#3498db',
    'S5': '#8e44ad'
}

fig_phase = go.FigureWidget()
fig_phase.update_layout(
    template='plotly_white', height=400,
    barmode='group',
    xaxis_title='Phase',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=60, b=60)
)

def update_phase_plot(*args):
    selected_singers = list(singer_select.value)
    selected_phases = list(phase_select.value)
    metric = metric_dropdown.value
    metric_label = dict(mean_hr='Mean HR (bpm)', sd_hr='SD (bpm)', min_hr='Min HR (bpm)', max_hr='Max HR (bpm)')[metric]
    
    with fig_phase.batch_update():
        fig_phase.data = []
        
        for s in selected_singers:
            vals = []
            labels = []
            errors = []
            for p in selected_phases:
                row = phase_sum[(phase_sum['session'] == s) & 
                                (phase_sum['subject'] == s) & 
                                (phase_sum['phase'] == p)]
                if len(row) > 0:
                    vals.append(row[metric].values[0])
                    errors.append(row['sd_hr'].values[0] if metric == 'mean_hr' else 0)
                else:
                    vals.append(0)
                    errors.append(0)
                labels.append(phase_labels[p])
            
            label = f"{singer_info[s][0]} ({singer_info[s][1]})"
            error_y = dict(type='data', array=errors, visible=True) if metric == 'mean_hr' else None
            fig_phase.add_trace(go.Bar(
                name=label,
                x=labels, y=vals,
                marker_color=singer_plot_colors[s],
                error_y=error_y,
                hovertemplate='%{y:.1f}<extra>' + label + '</extra>'
            ))
        
        fig_phase.layout.yaxis.title = metric_label
        fig_phase.layout.title = f'Singer {metric_label} by Phase'

for w in [singer_select, phase_select, metric_dropdown]:
    w.observe(update_phase_plot, 'value')

controls2 = widgets.HBox([
    singer_select,
    phase_select,
    metric_dropdown
])

display(controls2)
display(fig_phase)
update_phase_plot()

---
## Session Overlay

Compare multiple singers' HR traces on the same timeline. Useful for seeing how different singers' stress responses diverge. All traces are aligned to elapsed time from the sync clap.

In [ ]:
# Session overlay — multiple singers on one plot

overlay_singers = widgets.SelectMultiple(
    options=[(f"{singer_info[s][0]} ({singer_info[s][1]})", s) for s in ['S1','S2','S3','S4','S5']],
    value=['S2','S3','S4'],
    description='Singers:',
    rows=5,
    style={'description_width': 'initial'}
)

overlay_smoothing = widgets.IntSlider(
    value=5, min=1, max=30, step=1,
    description='Smoothing (s):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

normalize_toggle = widgets.Checkbox(
    value=False,
    description='Normalize to pre-singing baseline (show change from baseline)',
    indent=False,
    style={'description_width': 'initial'}
)

fig_overlay = go.FigureWidget()
fig_overlay.update_layout(
    template='plotly_white', height=450,
    xaxis_title='Elapsed Time (seconds)',
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    margin=dict(t=60, b=60)
)

def update_overlay(*args):
    selected = list(overlay_singers.value)
    window = overlay_smoothing.value
    normalize = normalize_toggle.value
    
    with fig_overlay.batch_update():
        fig_overlay.data = []
        
        for s in selected:
            sess = hr[(hr['session'] == s) & (hr['subject'] == s)].copy()
            y = sess['hr_bpm'].copy()
            
            if normalize:
                baseline = sess[sess['phase'] == 'pre_singing']['hr_bpm'].mean()
                y = y - baseline
            
            if window > 1:
                y = y.rolling(window, center=True, min_periods=1).mean()
            
            label = f"{singer_info[s][0]} ({singer_info[s][1]})"
            fig_overlay.add_trace(go.Scatter(
                x=list(sess['elapsed_s']), y=list(y),
                mode='lines',
                name=label,
                line=dict(color=singer_plot_colors[s], width=2),
                hovertemplate='%{y:.0f} bpm<extra>' + label + '</extra>'
            ))
        
        if normalize:
            fig_overlay.add_hline(y=0, line_dash='dash', line_color='black', line_width=1)
            fig_overlay.layout.yaxis.title = 'HR Change from Baseline (bpm)'
            fig_overlay.layout.title = 'Normalized HR Trajectories'
        else:
            fig_overlay.layout.yaxis.title = 'Heart Rate (bpm)'
            fig_overlay.layout.title = 'Singer HR Overlay'

for w in [overlay_singers, overlay_smoothing, normalize_toggle]:
    w.observe(update_overlay, 'value')

controls3 = widgets.VBox([
    overlay_singers,
    overlay_smoothing,
    normalize_toggle
])

display(controls3)
display(fig_overlay)
update_overlay()

---
## Phase Summary Table

Filter the full phase summary data. Select subjects and phases to narrow the view.

In [ ]:
# Interactive phase summary table

table_session = widgets.SelectMultiple(
    options=[(f"{singer_info[s][0]} ({singer_info[s][1]})", s) for s in ['S1','S2','S3','S4','S5']],
    value=['S1','S2','S3','S4','S5'],
    description='Sessions:',
    rows=5,
    style={'description_width': 'initial'}
)

table_subject = widgets.SelectMultiple(
    options=[('Singers only', 'singers'), ('Panelists only', 'panelists'), ('All', 'all')],
    value=['singers'],
    description='Show:',
    rows=3,
    style={'description_width': 'initial'}
)

table_output = widgets.Output()

def update_table(*args):
    sessions = list(table_session.value)
    show = list(table_subject.value)
    
    mask = phase_sum['session'].isin(sessions)
    
    if 'all' not in show:
        subjects = []
        if 'singers' in show:
            subjects += ['S1','S2','S3','S4','S5']
        if 'panelists' in show:
            subjects += ['F1','F3']
        # For singers, only show rows where subject matches session
        singer_mask = (phase_sum['subject'].isin(['S1','S2','S3','S4','S5'])) & (phase_sum['subject'] == phase_sum['session'])
        panelist_mask = phase_sum['subject'].isin(['F1','F3'])
        if 'singers' in show and 'panelists' in show:
            subject_mask = singer_mask | panelist_mask
        elif 'singers' in show:
            subject_mask = singer_mask
        else:
            subject_mask = panelist_mask
        mask = mask & subject_mask
    
    df = phase_sum[mask].copy()
    df['phase'] = pd.Categorical(df['phase'], categories=phase_order, ordered=True)
    df = df.sort_values(['session', 'subject', 'phase'])
    df['phase_label'] = df['phase'].map(phase_labels)
    
    display_cols = ['session', 'subject', 'phase_label', 'mean_hr', 'sd_hr', 'min_hr', 'max_hr', 'duration_s']
    df_display = df[display_cols].rename(columns={
        'phase_label': 'Phase', 'mean_hr': 'Mean HR', 'sd_hr': 'SD',
        'min_hr': 'Min', 'max_hr': 'Max', 'duration_s': 'Duration (s)',
        'session': 'Session', 'subject': 'Subject'
    })
    
    with table_output:
        table_output.clear_output(wait=True)
        display(df_display.style.format({'Mean HR': '{:.1f}', 'SD': '{:.1f}'}).hide(axis='index'))

for w in [table_session, table_subject]:
    w.observe(update_table, 'value')

display(widgets.HBox([table_session, table_subject]))
display(table_output)
update_table()

---

### Quick Reference

| Singer | Pre-Singing HR | Song 1 HR | Song 2 HR | Reactivity | Activation Gap (Song 2) | Notes |
|--------|---------------|-----------|-----------|------------|------------------------|-------|
| S1 (M) | 128 | 144 | 154 | +16 | +56 | Steady climber |
| S2 (F) | 128 | 160 | 147 | +32 | +54 | Steepest ramp-up |
| S3 (F) | 126 | 150 | 152 | +24 | +59 | Moderate, steady |
| S4 (F) | 91 | 106 | 90 | +15 | −5 | Outlier — pre-existing relationship with F3 |
| S5 (M) | 143 | 167 | 170 | +24 | +80 | Highest absolute HR |

*All values are mean HR in bpm. Activation gap = singer − mean(F1, F3) during song 2.*